<a href="https://colab.research.google.com/github/JuanBailke/Transfer-Learning-DIO/blob/main/Transfer_Learning_DIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline

import os
os.environ["KERAS_BACKEND"] = "tensorflow"

import random
import numpy as np
import keras

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Model

In [2]:
# Baixa o ZIP oficial
!curl -L -o caltech-101.zip "https://data.caltech.edu/records/mzrjq-6wc02/files/caltech-101.zip?download=1"

# Descompacta o ZIP em pasta local
!unzip caltech-101.zip

# Extrai o .tar.gz que está dentro da pasta caltech-101
!tar -xf caltech-101/101_ObjectCategories.tar.gz

# Remove o ZIP
!rm caltech-101.zip

# Lista as pastas
!ls


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   476  100   476    0     0    935      0 --:--:-- --:--:-- --:--:--   935
100  131M  100  131M    0     0  22.3M      0  0:00:05  0:00:05 --:--:-- 29.4M
Archive:  caltech-101.zip
   creating: caltech-101/
  inflating: __MACOSX/._caltech-101  
  inflating: caltech-101/101_ObjectCategories.tar.gz  
  inflating: __MACOSX/caltech-101/._101_ObjectCategories.tar.gz  
  inflating: caltech-101/show_annotation.m  
  inflating: __MACOSX/caltech-101/._show_annotation.m  
  inflating: caltech-101/Annotations.tar  
  inflating: __MACOSX/caltech-101/._Annotations.tar  
101_ObjectCategories  caltech-101  __MACOSX  sample_data


In [3]:
root = '101_ObjectCategories'
exclude = ['BACKGROUND_Google', 'Motorbikes', 'airplanes', 'Faces_easy', 'Faces']
train_split, val_split = 0.7, 0.15

categories = [x[0] for x in os.walk(root) if x[0]][1:]
categories = [c for c in categories if c not in [os.path.join(root, e) for e in exclude]]

print(categories)

['101_ObjectCategories/panda', '101_ObjectCategories/kangaroo', '101_ObjectCategories/electric_guitar', '101_ObjectCategories/schooner', '101_ObjectCategories/lotus', '101_ObjectCategories/elephant', '101_ObjectCategories/pigeon', '101_ObjectCategories/beaver', '101_ObjectCategories/umbrella', '101_ObjectCategories/car_side', '101_ObjectCategories/cannon', '101_ObjectCategories/joshua_tree', '101_ObjectCategories/helicopter', '101_ObjectCategories/buddha', '101_ObjectCategories/llama', '101_ObjectCategories/pizza', '101_ObjectCategories/dolphin', '101_ObjectCategories/yin_yang', '101_ObjectCategories/anchor', '101_ObjectCategories/cougar_body', '101_ObjectCategories/gramophone', '101_ObjectCategories/octopus', '101_ObjectCategories/grand_piano', '101_ObjectCategories/garfield', '101_ObjectCategories/platypus', '101_ObjectCategories/stegosaurus', '101_ObjectCategories/scissors', '101_ObjectCategories/dragonfly', '101_ObjectCategories/Leopards', '101_ObjectCategories/windsor_chair', '101

In [4]:
# helper function to load image and return it and input vector
def get_image(path):
    img = image.load_img(path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [5]:
data = []
for c, category in enumerate(categories):
    images = [os.path.join(dp, f) for dp, dn, filenames
              in os.walk(category) for f in filenames
              if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
    for img_path in images:
        img, x = get_image(img_path)
        data.append({'x':np.array(x[0]), 'y':c})

# count the number of classes
num_classes = len(categories)

In [6]:
random.shuffle(data)

In [7]:
idx_val = int(train_split * len(data))
idx_test = int((train_split + val_split) * len(data))
train = data[:idx_val]
val = data[idx_val:idx_test]
test = data[idx_test:]

In [8]:
x_train, y_train = np.array([t["x"] for t in train]), [t["y"] for t in train]
x_val, y_val = np.array([t["x"] for t in val]), [t["y"] for t in val]
x_test, y_test = np.array([t["x"] for t in test]), [t["y"] for t in test]
print(y_test)

[72, 73, 3, 62, 92, 8, 73, 83, 67, 65, 85, 81, 13, 84, 90, 73, 88, 24, 22, 12, 57, 14, 48, 20, 71, 62, 9, 64, 86, 68, 42, 70, 68, 46, 83, 79, 93, 43, 27, 14, 35, 10, 13, 64, 11, 92, 43, 17, 87, 42, 93, 90, 28, 14, 14, 85, 46, 56, 12, 48, 42, 43, 96, 49, 91, 22, 47, 25, 73, 17, 24, 63, 1, 73, 61, 66, 43, 54, 59, 8, 71, 65, 82, 69, 44, 85, 50, 49, 82, 73, 3, 28, 46, 93, 93, 14, 21, 85, 89, 8, 5, 62, 37, 50, 85, 84, 32, 84, 30, 84, 89, 77, 62, 52, 63, 92, 1, 25, 9, 9, 85, 33, 79, 68, 48, 8, 5, 72, 95, 14, 11, 11, 25, 53, 46, 73, 53, 56, 9, 60, 19, 31, 32, 81, 18, 81, 14, 26, 31, 33, 17, 44, 15, 17, 71, 37, 27, 51, 50, 73, 27, 2, 48, 65, 42, 93, 28, 92, 57, 83, 72, 20, 8, 57, 5, 76, 75, 1, 57, 61, 32, 18, 0, 37, 43, 46, 62, 9, 85, 93, 2, 28, 16, 58, 70, 12, 61, 48, 78, 60, 2, 28, 13, 70, 92, 49, 50, 13, 39, 76, 3, 13, 73, 26, 51, 73, 57, 23, 46, 59, 82, 91, 46, 15, 60, 23, 59, 91, 51, 8, 18, 68, 87, 24, 62, 54, 12, 37, 46, 30, 43, 50, 71, 73, 62, 93, 8, 78, 19, 73, 73, 47, 28, 68, 50, 18, 

In [ ]:
# normalize data
x_train = x_train.astype('float32') / 255.
x_val = x_val.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

# convert labels to one-hot vectors
y_train = keras.utils.to_categorical(y_train, num_classes)
y_val = keras.utils.to_categorical(y_val, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
print(y_test.shape)

In [ ]:
# summary
print("finished loading %d images from %d categories"%(len(data), num_classes))
print("train / validation / test split: %d, %d, %d"%(len(x_train), len(x_val), len(x_test)))
print("training data shape: ", x_train.shape)
print("training labels shape: ", y_train.shape)


In [ ]:
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(root) for f in filenames if os.path.splitext(f)[1].lower() in ['.jpg','.png','.jpeg']]
idx = [int(len(images) * random.random()) for i in range(8)]
imgs = [image.load_img(images[i], target_size=(224, 224)) for i in idx]
concat_image = np.concatenate([np.asarray(img) for img in imgs], axis=1)
plt.figure(figsize=(16,4))
plt.imshow(concat_image)

In [ ]:
# build the network
model = Sequential()
print("Input dimensions: ",x_train.shape[1:])

model.add(Conv2D(32, (3, 3), input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation('relu'))

model.add(Dropout(0.5))

model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

In [ ]:
# compile the model to use categorical cross-entropy loss function and adadelta optimizer
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=128,
                    epochs=10,
                    validation_data=(x_val, y_val))


In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

In [ ]:
loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
vgg = keras.applications.VGG16(weights='imagenet', include_top=True)
vgg.summary()

In [ ]:
# make a reference to VGG's input layer
inp = vgg.input

# make a new softmax layer with num_classes neurons
new_classification_layer = Dense(num_classes, activation='softmax')

# connect our new layer to the second to last layer in VGG, and make a reference to it
out = new_classification_layer(vgg.layers[-2].output)

# create a new network between inp and out
model_new = Model(inp, out)


In [ ]:
# make all layers untrainable by freezing weights (except for last layer)
for l, layer in enumerate(model_new.layers[:-1]):
    layer.trainable = False

# ensure the last layer is trainable/not frozen
for l, layer in enumerate(model_new.layers[-1:]):
    layer.trainable = True

model_new.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model_new.summary()

In [ ]:
history2 = model_new.fit(x_train, y_train,
                         batch_size=128,
                         epochs=10,
                         validation_data=(x_val, y_val))


In [ ]:
fig = plt.figure(figsize=(16,4))
ax = fig.add_subplot(121)
ax.plot(history.history["val_loss"])
ax.plot(history2.history["val_loss"])
ax.set_title("validation loss")
ax.set_xlabel("epochs")

ax2 = fig.add_subplot(122)
ax2.plot(history.history["val_acc"])
ax2.plot(history2.history["val_acc"])
ax2.set_title("validation accuracy")
ax2.set_xlabel("epochs")
ax2.set_ylim(0, 1)

plt.show()

In [ ]:
loss, accuracy = model_new.evaluate(x_test, y_test, verbose=0)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

In [ ]:
img, x = get_image('101_ObjectCategories/airplanes/image_0003.jpg')
probabilities = model_new.predict([x])